In [1]:
# 使用bioes标注体系标注数据，然后训练模型
import tensorflow as tf
import bert4keras
import keras
import os
print(tf.__version__)
print(keras.__version__)
print(bert4keras.__version__)
import numpy as np
from bert4keras.backend import keras, K
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import ViterbiDecoder, to_array
from bert4keras.layers import ConditionalRandomField
from keras.layers import Dense,Flatten
from keras.models import Model
from tqdm import tqdm
from utils import utils
from imp import reload
reload(utils)

Using TensorFlow backend.


2.2.0
2.3.1
0.7.7


ImportError: cannot import name 'to_array'

In [3]:
train_data = utils.load_data('../data/train_bioes.txt')
valid_data = utils.load_data('../data/val_bioes.txt')

In [4]:
ret_train_lst = []
for data in train_data:
    ret = 0
    for char,_ in data:
        ret += len(char)
    ret_train_lst.append(ret)
ret_val_lst = []
for data in valid_data:
    ret = 0
    for char,_ in data:
        ret += len(char)
    ret_val_lst.append(ret)

In [5]:
print(max(ret_train_lst),min(ret_train_lst))
print(max(ret_val_lst),min(ret_val_lst))
print(len(ret_train_lst))
print(len(ret_val_lst))

300 6
299 6
2746
278


In [60]:
# 预训练模型的超参数
maxlen = 300
epochs = 15
batch_size = 24
bert_layers = 12
learing_rate = 1e-5 
crf_lr_multiplier = 1000 
rnn_lr_multiplier = 1000
# bert配置,这里需要自己指定预训练模型的路径
config_path = '../model/chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '../model/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '../model/chinese_L-12_H-768_A-12/vocab.txt'
# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

# 类别映射
labels = [
    'position',
    'name',
    'organization',
    'company',
    'address',
    'movie',
    'game',
    'government',
    'scene',
    'book',
    'mobile',
    'email',
    'QQ',
    'vx',
]

# 0 表示 'O'
# 其他数字表示对应的 B 和 I
id2label = dict(enumerate(labels))
label2id = {j: i for i, j in id2label.items()}
num_labels = len(labels) * 4 + 1

In [7]:
# id2label
label2id

{'position': 0,
 'name': 1,
 'organization': 2,
 'company': 3,
 'address': 4,
 'movie': 5,
 'game': 6,
 'government': 7,
 'scene': 8,
 'book': 9,
 'mobile': 10,
 'email': 11,
 'QQ': 12,
 'vx': 13}

In [19]:
model = build_transformer_model(
    config_path,
    checkpoint_path,
)

output_layer = 'Transformer-%s-FeedForward-Norm' % (bert_layers - 1)
output = model.get_layer(output_layer).output
output_for_crf = Dense(num_labels)(output)

In [25]:
# 使用crf进行训练
CRF = ConditionalRandomField(lr_multiplier=crf_lr_multiplier)
output_crf = CRF(output_for_crf)

model = Model(model.input, [output_crf])
model.summary()

model.compile(
    loss=[CRF.sparse_loss,],
    optimizer=Adam(learing_rate),
    metrics=[CRF.sparse_accuracy]
)

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]              
___________________________________________________________________________________________

In [63]:
# # 使用softmax损失函数进行训练
# model = build_transformer_model(
#     config_path,
#     checkpoint_path,
# )

# output_layer = 'Transformer-%s-FeedForward-Norm' % (bert_layers - 1)
# output = model.get_layer(output_layer).output
# # print('output:',output.shape) # (None, None, 768)  [batch_size,seq_len,hidden_size],这里768=12*64(12个head，每个head有64个隐向量)
# # [batch_size,seq_len,hidden_size]
# # output = Dense(num_labels)(output)
# # 取出[CLS]对应的向量用来做分类
# # note:这里
# output = Flatten()(output)
# # (None, 768)
# # print('output:',output.shape)
# output_for_softmax = Dense(units=num_labels,activation='softmax')(output)
# # print('output_for_softmax:',output_for_softmax.shape)  #(None, 57)

# model = Model(model.input, output_for_softmax)
# model.summary()

# model.compile(
#     loss='categorical_crossentropy',
#     optimizer=Adam(learing_rate),
#     metrics=['accuracy']
# )

TypeError: Layer flatten_7 does not support masking, but was passed an input_mask: Tensor("Transformer-11-FeedForward-Add_28/All:0", shape=(None, None), dtype=bool)

In [ ]:
class data_generator(DataGenerator):
    """
    数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        # is_end  是否到 epoch 的末尾， item 对应的 train_data[i]
        for is_end, item in self.sample(random):
#             print('item:',item) ['布莱克本', 'organization'], ['对', 'O']
            # vocab.txt 从 0 开始计数，0:[pad] 1-99:[unused*] 100:[UNK] [CLS]:101
            # token_ids:101 ，label：[0]在词表中0代表的是pad
            token_ids, labels = [tokenizer._token_start_id], [0]
            for token, token_label in item:
#                 print('token, token_label:',token, token_label)
                # 得到除了 [CLS] [SEP] 之外的每一个词的 ids
                # e.g '全国青联委员经纪人' -> [1059, 1744, 7471, 5468, 1999, 1447, 5307, 5279, 782]
                # [CLS]word1 word2,...,word_n[SEP]                 
                w_token_ids = tokenizer.encode(token)[0][1:-1]
                if len(token_ids) + len(w_token_ids) < maxlen:
                    # e.g [101, 1059, 1744, 7471, 5468, 1999, 1447, 5307, 5279, 782]
                    # 把 [CLS] 加在前面
                    token_ids += w_token_ids
                    # BIOES
                    if token_label == 'O':
                        # 如果是 'O' 则标签是 0
                        labels += [0] * len(w_token_ids)
                    else:
                        # 如果不是 'O' 生成 label2id 对应的标签
                        # e.g '全国青联委员经纪人' -> [0, 1, 2, 2, 2, 2, 2, 2, 2, 2] 注意：第一个 [CLS] 默认是 0
                        token_len = len(w_token_ids)
                        if token_len == 1:
                            S = label2id[token_label] * 4 + 4 
                            labels += ([S])
                        elif token_len == 2:
                            B = label2id[token_label] * 4 + 1
                            E = label2id[token_label] * 4 + 3
                            labels += ([B] + [E])
                        else:
                            B = label2id[token_label] * 4 + 1
                            I = label2id[token_label] * 4 + 2
                            E = label2id[token_label] * 4 + 3
                            labels += ([B] + [I] * (len(w_token_ids) - 2)+[E])
#                             labels += ([B] + [I] * (len(w_token_ids) - 1))
#                     print('')
#                         print('label:',token_label, 'label2id:',label2id[token_label])
#                         print('labels:',labels)
                else:
                    # 超过 maxlen 长度的部分被舍弃掉
                    break
#                 print('+' * 100)
            # 加上尾巴 [SEP]:102
            token_ids += [tokenizer._token_end_id]
            # 尾巴的 label 也是 0
            labels += [0]
            # 生成 segment_ids ，NER 只用了一个句子，所以这里都是 0，这里是数字0，而不是字母O
            segment_ids = [0] * len(token_ids)
            # 把样本组装成 batch
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append(labels)
            # 如果凑齐了 batchsize 个，或者到 epoch 的最后一个，那么就返回。
            if len(batch_token_ids) == self.batch_size or is_end:
                # 进行 padding 操作
                # [bs,seq] 默认按照 bs 中最大的长度进行填充，保证每个 bs 的长度是一致，用 0 填充
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []



class NamedEntityRecognizer(ViterbiDecoder):
    """
    命名实体识别器
    """
    def recognize(self, text):
        tokens = tokenizer.tokenize(text)
        mapping = tokenizer.rematch(text, tokens)
        token_ids = tokenizer.tokens_to_ids(tokens)
        segment_ids = [0] * len(token_ids)
        token_ids, segment_ids = to_array([token_ids], [segment_ids])
        # [batch_size,seq_len,label]==>[1,80,57]
        # 这里就输入了一个句子，所以batch_size = 1,
        # model.predict([token_ids, segment_ids])[0] 这里取出的是后面两个维度 [seq_len,label],label的个数是确定的，就是上面的4* __ +1
        nodes = model.predict([token_ids, segment_ids])[0]
        # dp解码寻找最优路径
        # 长度为 seq 的向量，labels 每个值表示标签值
        labels = self.decode(nodes)
        entities, starting = [], False
        for i, label in enumerate(labels):
            if label > 0: # BIES
                if label % 4 == 1:
                    # 对应的B，这里和上面的 BIOES 的下标对应
                    # 有了 B 意味着我们开始预测实体了，所以starting = True，就需要把这个预测结果放到对应的
                    # 集合中，
                    starting = True
                    # 把对应的下标位置 i,对应的标签存下来
                    entities.append([[i], id2label[(label - 1) // 4]])
                elif label%4 == 0: # 对应的S,预测为S,后面就不需要预测了，所以 starting = False
                    starting = False
                    entities.append([[i], id2label[(label - 4) // 4]])
                elif starting:
                    # 预测的结果不是 B,不是S，只能是I,E,把当前实体信息拿出来，把对应的角标存下来
                    # entities的结构：[[[i], id2label[(label - 1) // 4]],[[[i], id2label[(label - 1) // 4]]]]
                    # entities[-1][0]即 [i]这个部分
                    entities[-1][0].append(i)
                else:
                    starting = False
            else:
                starting = False
        # 把预测的结果还原成文本形式
        # 
        return [(text[mapping[w[0]][0]:mapping[w[-1]][-1] + 1], l) for w, l in entities]

# len(K.eval(CRF.trans)):标签类别数
# 除了 0 ，其余标签都是 non_starts,non_ends
NER = NamedEntityRecognizer(trans=K.eval(CRF.trans), starts=[0], ends=[0])

tp = defaultdict(int)
predict_true = defaultdict(int)
gt = defaultdict(int)
# 各个类别p，r值的存放
pr_dict = defaultdict(dict)
wrong_dict = defaultdict(list)

def calc_F1_for_every_cls(R, T):
    """R 预测标签，T真实标签 ，分别统计标签不匹配，entity不匹配，以及二者都不匹配的情况
    """
    for r_entity, r_label in R:
        predict_true[r_label] += 1
    for t_entity, t_label in T:
        gt[t_label] += 1
    for r_entity, r_label in R:
        for t_entity, t_label in T:
            if r_entity == t_entity and r_label == t_label:
                tp[r_label] += 1
            # 把预测错误的数据保存下来，
            elif r_entity != t_entity and r_label == t_label:
                for label in labels:
                    if label == r_label:
                        wrong_dict['entity_wrong_'+label].append([(r_entity, r_label),(r_entity, r_label)])
            elif r_entity == t_entity and r_label != t_label:
                for label in labels:
                    if label==r_label:
                        wrong_dict['label_wrong_'+label].append([(r_entity, r_label),(r_entity, r_label)])
            else:
                # 标签，entity都预测错误的就没办法分类了
                wrong_dict['all_wrong'].append(([(r_entity, r_label),(r_entity, r_label)]))
                
    with open('wrong_predict.txt','a',encoding='utf-8') as f:
        for label in labels:
            for item in wrong_dict['entity_wrong_'+label]:
                f.write(str(item)+'\n')
        f.write('\n')
        
        for label in labels:
            for item in wrong_dict['label_wrong_'+label]:
                f.write(str(item)+'\n')
        f.write('\n')
        
        for item in wrong_dict['all_wrong']:
            f.write(str(item)+'\n')
        f.write('\n\n')
                
def evaluate(data):
    """评测函数
    """
    X, Y, Z = 1e-10, 1e-10, 1e-10
    
    for d in tqdm(data):
        text = ''.join([i[0] for i in d])
        R = set(NER.recognize(text)) # 预测
        T = set([tuple(i) for i in d if i[1] != 'O']) #真实
        # 计算每个类别的p,r值，并把预测错误的结果保存下来，同时统计每个类别的数据量，数据不平衡问题如何处理
        # 一个样本里面预测的结果
        # R: {('美国', 'address'), ('War3', 'game'), ('SC2', 'game'), ('暴雪', 'company'), ('DotA', 'game')}
        # T: {('美国', 'address'), ('War3', 'game'), ('SC2', 'game'), ('暴雪', 'company'), ('DotA', 'game')}
        calc_F1_for_every_cls(R,T)
        X += len(R & T) 
        Y += len(R) 
        Z += len(T)
    
    for label in labels:
        if predict_true[label]!=0:
          pr_dict['precision'][label] = round(tp[label] / predict_true[label],5)
        if gt[label]!=0:
          pr_dict['recall'][label] = round(tp[label] / gt[label],5)
    precision, recall =  X / Y, X / Z
    f1 = 2*precision*recall/(precision+recall)
    return f1, precision, recall,pr_dict


class Evaluator(keras.callbacks.Callback):
    def __init__(self,valid_data):
        self.best_val_f1 = 0
        self.valid_data = valid_data
        print('valid_data,',len(valid_data))

    def on_epoch_end(self, epoch, logs=None):
#         print('epoch:',epoch)
        trans = K.eval(CRF.trans)
        NER.trans = trans
#         print(NER.trans)
        f1, precision, recall,pr_dict = evaluate(self.valid_data)
        # 保存最优
        if f1 >= self.best_val_f1:
            self.best_val_f1 = f1
            model.save_weights('../model/best_model_bioes.weights')
        print(
            'valid:  f1: %.5f, precision: %.5f, recall: %.5f, best f1: %.5f' %
            (f1, precision, recall, self.best_val_f1)
        )
        print('precision for every class:',pr_dict['precision'])
        print('recall for every class:',pr_dict['recall'])



evaluator = Evaluator(valid_data)
train_generator = data_generator(train_data, batch_size)
model.fit_generator(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    callbacks=[evaluator]
)